In [392]:
import pyfabrik
from vectormath import Vector2

import pygame
import numpy as np
import sys
import math
%gui qt

ERROR:root:
    Could not load requested Qt binding. Please ensure that
    PyQt4 >= 4.7, PyQt5, PyQt6, PySide >= 1.0.3, PySide2, or
    PySide6 is available, and only one is imported per session.

    Currently-imported Qt library:                              None
    PyQt5 available (requires QtCore, QtGui, QtSvg, QtWidgets): False
    PyQt6 available (requires QtCore, QtGui, QtSvg, QtWidgets): False
    PySide2 installed:                                          False
    PySide6 installed:                                          False
    Tried to load:                                              ['pyqtdefault', 'pyqt6', 'pyside6', 'pyqt5', 'pyside2']
    


### Algorithme témoin

In [393]:
initial_joint_positions = [Vector2(0, 0), Vector2(10, 0), Vector2(20, 0)]
tolerance = 0.01

# Initialize the Fabrik class (Fabrik, Fabrik2D or Fabrik3D)
fab = pyfabrik.Fabrik3D(initial_joint_positions, tolerance)

fab.move_to(Vector2(20, 0))
fab.angles_deg # Holds [0.0, 0.0, 0.0]

fab.move_to(Vector2(60, 60)) # Return 249 as number of iterations executed
fab.angles_deg # Holds [43.187653094161064, 3.622882738369357, 0.0]

[43.187653094161064, 3.622882738369357, 0.0]

In [394]:
def calculer_coordonnees(longueurs, angle_radians):
    nodes = []
    x_prev, y_prev = 400, 600  

    for i in range(len(longueurs)):
        x = x_prev + longueurs[i] * math.cos(angle_radians[i])
        y = y_prev + longueurs[i] * math.sin(angle_radians[i])
        nodes.append([x, y])
        x_prev, y_prev = x, y  
    return nodes

In [395]:
def calculer_angles(coord_list):
    angles = []
    
    # Parcourir la liste de coordonnées
    for i in range(1, len(coord_list)):
        dx = coord_list[i][0] - coord_list[i-1][0]
        dy = coord_list[i][1] - coord_list[i-1][1]
        
        # Calculer l'angle entre les segments en radians
        if i == 1:
            # Pour le premier segment, l'angle est calculé par rapport à l'axe x
            angle = math.atan2(dy, dx)
        else:
            # Pour les segments suivants, l'angle est calculé par rapport au segment précédent
            dx_prev = coord_list[i-1][0] - coord_list[i-2][0]
            dy_prev = coord_list[i-1][1] - coord_list[i-2][1]
            angle = math.atan2(dy, dx) - math.atan2(dy_prev, dx_prev)
        
        # Convertir l'angle en degrés
        angle_deg = math.degrees(angle)
        
        # Ajouter l'angle à la liste des angles
        angles.append(angle_deg)
    
    return angles

### Algorithme maison

In [396]:
class Arm:

    def __init__(self, base, lengths, angles):
        self.base = base
        self.lengths = lengths
        self.angles = angles
        self.coordinates = [base]
        #self.coordinates = [base, Node(0,100), Node(0, 200), Node(0, 300), Node(0, 400)] # Ne marche qu'avec un nombre pair de coords ??
        for node in calculer_coordonnees(lengths, angles):
            self.coordinates.append(node)
        
    
    def reach(self, head, tail, tgt):
        # calculate the current length
        c_dx = tail[0] - head[0]
        c_dy = tail[1] - head[1]
        c_dist = math.sqrt(c_dx * c_dx + c_dy * c_dy)
        
        # calculate the stretched length
        s_dx = tail[0] - tgt[0]
        s_dy = tail[1] - tgt[1]
        s_dist = math.sqrt(s_dx * s_dx + s_dy * s_dy)
        
        # If the stretched distance is zero, set scale to 1 to avoid division by zero
        if s_dist == 0:
            scale = 1
        else:
            scale = c_dist / s_dist
        
        # return the result
        return [
            # copy the target for the new head
            [tgt[0], tgt[1]],
            
            # scale the new tail based on distance from target
            [tgt[0] + s_dx * scale, tgt[1] + s_dy * scale]
        ]


    def update_angles(self, delta):
        angle_limits = [(-90, 90), (-90, 90), (-90, 90)]
        for i in range(len(delta)):
            new_angle = self.angles[i] + delta[i]
            min_angle, max_angle = angle_limits[i]
            self.angles[i] = max(min(new_angle, max_angle), min_angle)
    
    def jacobian(self):
        N = len(self.angles)

        matX = [0] * N
        line = [self.lengths[i] * math.sin(sum(self.angles[0:i+1])) for i in range(N)]              
        for i in range(N):
            matX[i] = -1 * sum(line)
            line = line[1:]
            
        matY = [0] * N
        line = [self.lengths[i] * math.cos(sum(self.angles[0:i+1])) for i in range(N)]                            
        for i in range(N):
            matY[i] = sum(line)
            line = line[1:]
        
        return np.array([matX, matY])
    
    def pseudo_J_inv(self):
        J = self.jacobian()
        return np.linalg.pinv(J)

    def calculateError(self, target):
        # Calcul de l'erreur de position
        target_rel = (target[0] - self.base[0], self.base[1] - target[1])

        current = 0
        cumul = [0, 0]
        for i in range(len(self.angles)):
            current += self.angles[i]
            cumul[0] += self.lengths[i] * math.cos(current)
            cumul[1] += self.lengths[i] * math.sin(current)
        
        error_x = target_rel[0] - cumul[0]
        error_y = target_rel[1] - cumul[1]
        return np.array([error_x, error_y])

    def inverse_pseudo_jacobian(self, target):
        max_iteration = 100

        for _ in range(max_iteration):
            pseudo_inv_J = self.pseudo_J_inv()
            error = self.calculateError(target)

            # Calcul de la commande de vitesse
            delta_theta = 0.1 * np.dot(pseudo_inv_J, error)

            
            self.update_angles(delta_theta)
            
            current = 0
            cumul = [0, 0]
            for i in range(len(self.angles)):
                current += self.angles[i]
                cumul[0] += self.lengths[i] * math.cos(current)
                cumul[1] += self.lengths[i] * math.sin(current)
                self.joint_positions[i+1] = ((self.base[0]+cumul[0]),(self.base[1]-cumul[1]))

    
    def fabrik(self, target):
        # Forward reaching
        self.coordinates[-1] = target
        for i in range(len(self.coordinates) - 2, -1, -1):
            current_length = math.hypot(self.coordinates[i+1][0] - self.coordinates[i][0],
                                        self.coordinates[i+1][1] - self.coordinates[i][1])
            lambda_val = self.lengths[i] / current_length
            self.coordinates[i] = ((1 - lambda_val) * self.coordinates[i+1][0] + lambda_val * self.coordinates[i][0],
                                (1 - lambda_val) * self.coordinates[i+1][1] + lambda_val * self.coordinates[i][1])
            
        # Backward reaching
        self.coordinates[0] = (self.base[0], self.base[1])
        for i in range(len(self.coordinates) - 1):
            current_length = math.hypot(self.coordinates[i+1][0] - self.coordinates[i][0],
                                        self.coordinates[i+1][1] - self.coordinates[i][1])
            lambda_val = self.lengths[i] / current_length
            self.coordinates[i+1] = ((1 - lambda_val) * self.coordinates[i][0] + lambda_val * self.coordinates[i+1][0],
                                    (1 - lambda_val) * self.coordinates[i][1] + lambda_val * self.coordinates[i+1][1])


        #Update les angles
        self.angles = calculer_angles(self.coordinates)

In [397]:
class Screen:

    def __init__(self, arm, width, height):
        self.arm = arm
        self.width = width
        self.height = height

        # Initialisation de pygame
        pygame.init()
        self.window = pygame.display.set_mode((width, height))
        pygame.display.set_caption("Robot ARM")

    def draw(self):
        # Couleurs
        blanc = (255, 255, 255)
        noir = (0, 0, 0)
        red = (255, 0, 0)

        # Position initiale du cercle
        x_cercle, y_cercle = self.width // 2, self.height // 2
        rayon_cercle = 5

            # Effacement de l'écran
        self.window.fill(blanc)


        # Dessin du cercle

        for i in range (len(self.arm.coordinates)-1):
            if i == 0:
                pygame.draw.circle(self.window, red, self.arm.coordinates[i], rayon_cercle)  
            else:
                pygame.draw.circle(self.window, noir, self.arm.coordinates[i], rayon_cercle)
            pygame.draw.line(self.window, noir, self.arm.coordinates[i], self.arm.coordinates[i+1], rayon_cercle)
        pygame.draw.circle(self.window, noir, self.arm.coordinates[len(self.arm.coordinates)-1], rayon_cercle)
    

        # Rafraîchissement de l'écran
        pygame.display.flip()

In [398]:


base = [400,600]
arm = Arm(base, [100, 100, 100], [((4 * math.pi) / 3), ((11 * math.pi) / 6), ((1 * math.pi) / 6)])
screen = Screen(arm, 800, 600)

#arm = [Node(0,0), Node(0,100), Node(0, 200)]
# Dimensions de la fenêtre
largeur, hauteur = 800, 600
#fenetre = pygame.display.set_mode((largeur, hauteur))
#pygame.display.set_caption("Cercle qui suit la souris")



# Boucle principale

target = None

while True:
    for evenement in pygame.event.get():
        if evenement.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif evenement.type == pygame.MOUSEBUTTONUP:
              x_souris, y_souris = pygame.mouse.get_pos()
              target = [x_souris, y_souris]
  
  
    # Mise à jour de la position du cercle en fonction de la souris
    if target != None:
        arm.inverse_pseudo_jacobian(target)

    for i in range(len(arm.coordinates)):
            print(arm.coordinates[i])
    print(arm.angles)
    print()
        
    screen.draw()

# Nettoyage et fermeture
pygame.quit()
sys.exit()

(299.9984084838417, 0.9771935129766829)
(199.99893898922784, 0.6514623419844541)
(99.9994694946138, 0.32573117099222637)
(0, 0)
[-179.81336945643443, 0.0, 0.0]


[400, 600]
[349.99999999999994, 513.3974596215562]
[436.6025403784438, 463.39745962155615]
[523.2050807568877, 513.3974596215561]
[4.1887902047863905, 5.759586531581287, 0.5235987755982988]

[400, 600]
[349.99999999999994, 513.3974596215562]
[436.6025403784438, 463.39745962155615]
[523.2050807568877, 513.3974596215561]
[4.1887902047863905, 5.759586531581287, 0.5235987755982988]

[400, 600]
[349.99999999999994, 513.3974596215562]
[436.6025403784438, 463.39745962155615]
[523.2050807568877, 513.3974596215561]
[4.1887902047863905, 5.759586531581287, 0.5235987755982988]

[400, 600]
[349.99999999999994, 513.3974596215562]
[436.6025403784438, 463.39745962155615]
[523.2050807568877, 513.3974596215561]
[4.1887902047863905, 5.759586531581287, 0.5235987755982988]

[400, 600]
[349.99999999999994, 513.3974596215562]
[436.6025403784438, 463.39745962155615]
[523.2050807568877, 513.3974596215561]
[4.1887902047863905, 5.759586531581287, 0.5235987755982988]

[400, 600]
[349.99999999999994, 513.397459621

TypeError: 'tuple' object does not support item assignment

In [ ]:
class RobotArm:

    def __init__(self, base, arm_length):
        self.base = base
        self.angles = None
        self.arm_length = arm_length #Liste d'entier
        self.joint_positions = [base]

        #self.joint_positions = joint_positions #Liste de node

    #Position initiale du robot
    def initiate(self):

        self.angles = [0, 0, 0]
        
        for i in range(len(self.arm_length)):
            angle = math.radians(sum(self.angles[:i+1]))
            x = self.joint_positions[i].x + self.arm_length[i] * math.cos(angle)
            y = self.joint_positions[i].y - self.arm_length[i] * math.sin(angle)
            self.joint_positions.append(Node(x,y))


    #Mettre à jour les points
    def update_angles(self, delta):
        angle_limits = [(-90, 90), (-90, 90), (-90, 90)]
        for i in range(len(delta)):
            new_angle = self.angles[i] + delta[i]
            min_angle, max_angle = angle_limits[i]
            self.angles[i] = max(min(new_angle, max_angle), min_angle)
    
    def jacobian(self):
        N = len(self.angles)

        matX = [0] * N
        line = [self.arm_length[i] * math.sin(sum(self.angles[0:i+1])) for i in range(N)]              
        for i in range(N):
            matX[i] = -1 * sum(line)
            line = line[1:]
            
        matY = [0] * N
        line = [self.arm_length[i] * math.cos(sum(self.angles[0:i+1])) for i in range(N)]                            
        for i in range(N):
            matY[i] = sum(line)
            line = line[1:]
        
        return np.array([matX, matY])
    
    def pseudo_J_inv(self):
        J = self.jacobian()
        return np.linalg.pinv(J)

    def calculateError(self, target):
        # Calcul de l'erreur de position
        target_rel = Node(target.x - self.base.x, self.base.y - target.y)

        current = 0
        cumul = Node(0, 0)
        for i in range(len(self.angles)):
            current += self.angles[i]
            cumul.x += self.arm_length[i] * math.cos(current)
            cumul.y += self.arm_length[i] * math.sin(current)
        
        error_x = target_rel.x - cumul.x
        error_y = target_rel.y - cumul.y
        return np.array([error_x, error_y])

    def move_arm(self, target):
        max_iteration = 100

        for _ in range(max_iteration):
            pseudo_inv_J = self.pseudo_J_inv()
            error = self.calculateError(target)

            # Calcul de la commande de vitesse
            delta_theta = 0.1 * np.dot(pseudo_inv_J, error)

            
            self.update_angles(delta_theta)
            
            current = 0
            cumul = Node(0, 0)
            for i in range(len(self.angles)):
                current += self.angles[i]
                cumul.x += self.arm_length[i] * math.cos(current)
                cumul.y += self.arm_length[i] * math.sin(current)
                self.joint_positions[i+1] = Node((self.base.x+cumul.x),(self.base.y-cumul.y))

                    
    
    def draw_arm(self, screen):
        BLACK = (0,0,0)
        RED = (255, 0, 0)
        GREEN = (0, 255, 0)

        # Dessiner le bras robotique
        pygame.draw.circle(screen, RED, self.joint_positions[0].get_coordinates(), 10)
        for i in range(len(self.joint_positions)-1):
            pygame.draw.line(screen, BLACK, self.joint_positions[i].get_coordinates(), self.joint_positions[i+1].get_coordinates(), 5)
            pygame.draw.circle(screen, RED, (self.joint_positions[i+1].get_coordinates()), 10)

if __name__ == '__main__':
    # Initialisation de Pygame
    pygame.init()
    width, height = 600, 300
    window = pygame.display.set_mode((width, height))
    clock = pygame.time.Clock()

    # Couleurs
    WHITE = (255, 255, 255)

    # Paramètres du bras robotique
    lengths = [100,100,100]
    base = Node(0, 0)
    robot_arm = RobotArm(base,lengths)
    robot_arm.initiate()

    # Position initiale du bras robotique

    # Position cible

    target = None


    # Boucle principale
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Bouton gauche de la souris
                    target_pos = pygame.mouse.get_pos()
                    target = Node(target_pos[0], target_pos[1])

        # Effacer l'écran
        window.fill(WHITE)



        if(target!=None):
            
            robot_arm.move_arm(target)
            pygame.draw.circle(window, (0,255,0), target.get_coordinates(), 15)

        robot_arm.draw_arm(window)


    
        pygame.display.flip()

    pygame.quit()


In [ ]:
import math

def calculer_longueurs_angles(coord_list):
    longueurs = []
    angles = []
    
    # Parcourir la liste de coordonnées
    for i in range(1, len(coord_list)):
        # Calculer la longueur du segment entre les coordonnées actuelles et les précédentes
        dx = coord_list[i][0] - coord_list[i-1][0]
        dy = coord_list[i][1] - coord_list[i-1][1]
        longueur = math.sqrt(dx**2 + dy**2)
        longueurs.append(longueur)
        
        # Calculer l'angle entre les segments en radians
        if i == 1:
            # Pour le premier segment, l'angle est calculé par rapport à l'axe x
            angle = math.atan2(dy, dx)
        else:
            # Pour les segments suivants, l'angle est calculé par rapport au segment précédent
            dx_prev = coord_list[i-1][0] - coord_list[i-2][0]
            dy_prev = coord_list[i-1][1] - coord_list[i-2][1]
            angle = math.atan2(dy, dx) - math.atan2(dy_prev, dx_prev)
        
        # Convertir l'angle en degrés
        angle_deg = math.degrees(angle)
        
        # Ajouter l'angle à la liste des angles
        angles.append(angle_deg)
    
    return longueurs, angles

# Exemple d'utilisation
coord_list = [(0, 0), (100, 0), (200, 0)]
longueurs, angles = calculer_longueurs_angles(coord_list)

print("Longueurs des segments:", longueurs)
print("Angles entre les segments:", angles)


Longueurs des segments: [100.0, 100.0]
Angles entre les segments: [0.0, 0.0]
